In [14]:
from pathlib import Path
import pyvista as pv
import numpy as np
import matplotlib.colors as colors
import matplotlib.cm as cm
import subprocess
import os

filepath = Path('/home/simoneponcioni/Documents/01_PHD/03_Methods/HFE/02_CODE/src/hfe_utils/plot_strain_fields.ipynb')

In [15]:
def plot_midslice_vtk(vtkpath: Path, celldata: str):
    def custom_colormap():
        # Create a custom colormap
        cmap = colors.LinearSegmentedColormap.from_list(
            "transparent_to_red", [(0, 0, 0, 0), (1, 0, 0, 1)]
        )

        norm = colors.Normalize(vmin=np.min(clip_masked[celldata]), vmax=np.max(clip_masked[celldata]))
        color_mapper = cm.ScalarMappable(norm=norm, cmap=cmap)
        return color_mapper, cmap

    vtkpv = pv.read(vtkpath)
    sliced = vtkpv.clip(normal=[1,0,0], invert=True)

    clip_masked = sliced.threshold(
        [np.quantile(sliced.cell_data[celldata], 0.00), np.quantile(sliced.cell_data[celldata], 0.98)],
        scalars=celldata, invert=False)

    # color_mapper, cmap = custom_colormap()
    # clip_masked['IsovolumicDeformation_Colors'] = color_mapper.to_rgba(clip_masked[celldata])

    edges = sliced.extract_feature_edges(60)

    p = pv.Plotter(notebook=False, off_screen=True)
    p.add_mesh(edges, color='black', lighting=False)
    p.add_mesh(
        clip_masked,
        # scalars='IsovolumicDeformation_Colors',
        scalars='IsovolumicDeformation',
        # cmap=cmap,
        cmap='PuBu',
        show_edges=False,
        log_scale=False,
        scalar_bar_args={"title": celldata, "n_labels": 5}
    )
    p.view_yz()
    p.camera.roll += 180
    p.camera.SetParallelProjection(True)
    p.camera.zoom(1.4)
    p.show()
    name_with_celldata = vtkpath.stem + '_' + celldata
    vtk_parent_path = vtkpath.parent / 'midslice'
    vtk_parent_path.mkdir(parents=True, exist_ok=True)
    p.screenshot(vtk_parent_path / f'{name_with_celldata}.png')


# vtkpath = r'/home/simoneponcioni/Documents/01_PHD/03_Methods/HFE/04_SIMULATIONS/438_L_71_F/C0003104_04/Step-Compression_19_with_data.vtu'

vtudir = r'/home/simoneponcioni/Documents/01_PHD/04_Output-Reports-Presentations-Publications/HFE-RESULTS/strain-distribution/20240501/'
# patlib glob through all the vtu files

celldata = 'IsovolumicDeformation'
# celldata = 'SphericalCompression'
for vtkpath in Path(vtudir).rglob('*.vtu'):
    plot_midslice_vtk(Path(vtkpath), celldata)

In [16]:
sim_results_path = Path(r'/home/simoneponcioni/Documents/01_PHD/04_Output-Reports-Presentations-Publications/HFE-RESULTS/strain-distribution/20240501/midslice')
exp_results_path = Path(r'/home/simoneponcioni/Documents/01_PHD/04_Output-Reports-Presentations-Publications/HFE-RESULTS/strain-distribution/FRACTIB-publication/')

isovolumic_dict = {}
spherical_dict = {}
for sim_img in sim_results_path.rglob('*.png'):
    sim_name = sim_img.stem.split('_')[0]
    exp_img = exp_results_path / f'{sim_name}_Reg.png'
    
    if "IsovolumicDeformation" in sim_img.stem:
        isovolumic_dict[sim_img] = exp_img
    elif "SphericalCompression" in sim_img.stem:
        spherical_dict[sim_img] = exp_img

# Sorting dictionaries by their keys and storing them as dictionaries again
isovolumic_dict = dict(sorted(isovolumic_dict.items(), key=lambda item: item[0]))
spherical_dict = dict(sorted(spherical_dict.items(), key=lambda item: item[0]))

import pprint
pprint.pprint(isovolumic_dict)

{PosixPath('/home/simoneponcioni/Documents/01_PHD/04_Output-Reports-Presentations-Publications/HFE-RESULTS/strain-distribution/20240501/midslice/432_C0003103_IsovolumicDeformation.png'): PosixPath('/home/simoneponcioni/Documents/01_PHD/04_Output-Reports-Presentations-Publications/HFE-RESULTS/strain-distribution/FRACTIB-publication/432_Reg.png'),
 PosixPath('/home/simoneponcioni/Documents/01_PHD/04_Output-Reports-Presentations-Publications/HFE-RESULTS/strain-distribution/20240501/midslice/433_C0003114_IsovolumicDeformation.png'): PosixPath('/home/simoneponcioni/Documents/01_PHD/04_Output-Reports-Presentations-Publications/HFE-RESULTS/strain-distribution/FRACTIB-publication/433_Reg.png'),
 PosixPath('/home/simoneponcioni/Documents/01_PHD/04_Output-Reports-Presentations-Publications/HFE-RESULTS/strain-distribution/20240501/midslice/434_C0003111_IsovolumicDeformation.png'): PosixPath('/home/simoneponcioni/Documents/01_PHD/04_Output-Reports-Presentations-Publications/HFE-RESULTS/strain-dist

In [17]:
beamer_isovolumic = sim_results_path / 'beamer_isovolumic_deformation.tex'
beamer_spherical = sim_results_path / 'beamer_spherical_compression.tex'
for beamer_presentation, coupling_dict in zip([beamer_isovolumic, beamer_spherical], [isovolumic_dict, spherical_dict]):
    with open(beamer_presentation, 'w') as f:
        # Write the preamble
        f.write("\\documentclass{beamer}\n\\usepackage{graphicx}\n\\begin{document}\n")

        # Iterate over the dictionary
        for sim_img, exp_img in coupling_dict.items():
            # Replace underscores in the file paths
            sim_img_latex = str(sim_img).replace("_", "\\_")
            exp_img_latex = str(exp_img).replace("_", "\\_")
            
            sim_name = str(sim_img.name).replace("_", "\\_")
            exp_name = str(exp_img.name).replace("_", "\\_")

            # Write the LaTeX code for the slide
            f.write("\\begin{frame}\n")
            f.write("\\begin{columns}\n")
            
            f.write("\\begin{column}{0.5\\textwidth}\n")
            f.write("\\begin{figure}\n")
            f.write("\\includegraphics[width=\\textwidth]{" + sim_img_latex + "}\n")
            f.write("\\caption{" + sim_name + "}\n")  # Add caption for the sim_img
            f.write("\\end{figure}\n")
            f.write("\\end{column}\n")
            
            f.write("\\begin{column}{0.5\\textwidth}\n")
            f.write("\\begin{figure}\n")
            f.write("\\includegraphics[width=\\textwidth]{" + exp_img_latex + "}\n")
            f.write("\\caption{" + exp_name + "}\n")  # Add caption for the exp_img
            f.write("\\end{figure}\n")
            f.write("\\end{column}\n")
            
            f.write("\\end{columns}\n")
            f.write("\\end{frame}\n")

        # Write the end of the document
        f.write("\\end{document}\n")

    # Call pdflatex to compile the .tex file to a .pdf
    os.chdir(sim_results_path)
    subprocess.run(['pdflatex', str(beamer_presentation)], check=True)
    os.chdir(filepath.parent)

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode

(/home/simoneponcioni/Documents/01_PHD/04_Output-Reports-Presentations-Publicat
ions/HFE-RESULTS/strain-distribution/20240501/midslice/beamer_isovolumic_deform
ation.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/beamer/beamer.cls
Document Class: beamer 2022/01/21 v3.65 A class for typesetting presentations
(/usr/share/texlive/texmf-dist/tex/latex/beamer/beamerbasemodes.sty
(/usr/share/texlive/texmf-dist/tex/latex/etoolbox/etoolbox.sty)
(/usr/share/texlive/texmf-dist/tex/latex/beamer/beamerbasedecode.sty))
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifpdf.sty
(/usr/share/texlive/texmf-dist/tex/generic/iftex/iftex.sty))
(/usr/share/texlive/texmf-dist/tex/latex/beamer/beamerbaseoptions.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty))
(/usr/share